# 使用 Logistic 迴歸預測 Titanic 乘客的存活機率

郭耀仁

## 讀入資料

我們使用 `pandas` 套件的 `read_csv()` 方法讀入資料集。

In [8]:
import pandas as pd

# 讀入資料
url = "https://storage.googleapis.com/py_ds_basic/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 看看資料
titanic_train.head() # 發現 Cabin 有 NaN

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 了解變數的意義

到 [Data - Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data) 了解變數的意義。

In [5]:
# 描述性統計
titanic_train.describe() # 發現 Age 有 NaN

/Users/tonykuo/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 填補年齡的遺漏值

我們使用中位數來填補 Age 變數的遺漏值。

In [11]:
import numpy as np

age_median = np.nanmedian(titanic_train["Age"]) # 要忽略 NaN
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 建立 dummy variables

Logistic 迴歸必須要將類別變數轉換成 dummy variables。

In [9]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])
encoded_Pclass = label_encoder.fit_transform(titanic_train["Pclass"])

## 建立 X 與 y

**Scikit-learn** 完全使用矩陣運算。

$$y_i = X_i^Tw$$

In [26]:
titanic_X = pd.DataFrame([encoded_Pclass,
                          encoded_Sex,
                          new_Age
]).T

titanic_y = titanic_train["Survived"]

## 切分訓練與測試資料

利用 70%/30% 這個比例切分訓練與測試資料。

In [27]:
from sklearn.cross_validation import train_test_split

# 切分訓練與測試資料
train_X, test_X, train_y, test_y = train_test_split(titanic_X, titanic_y, test_size = 0.3)

## 建立 Logistic 迴歸模型

我們使用 **Sex**，**Pclass** 與 **Age** 來預測 **Survived**。

In [28]:
from sklearn import metrics, linear_model

# 建立模型
logistic_regr = linear_model.LogisticRegression()
logistic_regr.fit(train_X, train_y)

# 預測
test_y_predicted = logistic_regr.predict(test_X)

# 準確率
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)

0.817164179104


## 應用模型在沒有答案的資料

[Submission Instructions](https://www.kaggle.com/c/titanic/details/submission-instructions)

In [32]:
url = "https://storage.googleapis.com/py_ds_basic/kaggle_titanic_test.csv"
to_submit = pd.read_csv(url)

# 跟訓練資料作一樣的整理
encoded_Sex_to_submit = label_encoder.fit_transform(to_submit["Sex"])
encoded_Pclass_to_submit = label_encoder.fit_transform(to_submit["Pclass"])
age_median = np.nanmedian(to_submit["Age"]) # 要忽略 NaN
imputed_Age = np.where(to_submit["Age"].isnull(), age_median, to_submit["Age"])

to_submit_X = pd.DataFrame([encoded_Sex_to_submit,
                            encoded_Pclass_to_submit,
                            imputed_Age
]).T

# 預測
to_submit_y = logistic_regr.predict(to_submit_X)
print(to_submit_y)

# 準備要上傳的檔案
to_submit_dict = {
    "PassengerId": to_submit["PassengerId"],
    "Survived": to_submit_y
}
to_submit_df = pd.DataFrame(to_submit_dict)

# 輸出成 csv
to_submit_df.to_csv("to_submit.csv", index = False)

[0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 0 0 1
 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0
 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1
 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0
 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0
 1 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0
 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0
 1 0 0 0 1 0 0 1 0 0 0]
